# Wood density maping by random forest models in GEE

In [ ]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
from termcolor import colored # this is allocate colour and fonts type for the print title and text

In [2]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()
os.chdir('/Users/LeonidMoore/Desktop/ETH_Study/WoodDensityMapingProject')

In [3]:
# initialize the earth engine API
ee.Initialize()

In [4]:
# Load the composite for root shoot ratio analysis
compositeRaw = ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
# get the projection
stdProj = compositeRaw.projection()
forestAgeData = ee.Image("projects/crowtherlab/johan/Besnard_ForestAge")
print(colored('Band in Forest age data:\n', 'blue', attrs=['bold']),forestAgeData.bandNames().getInfo())
# we choose the band "forest_age_TC030" for following modeling
forestAge = forestAgeData.select(['forest_age_TC000']).reproject(crs=stdProj).rename('ForestAge')
# load the additional layers and uniform the projection
soilmoisture = ee.Image('users/haozhima95/wld_soil_moisture').reproject(crs=stdProj).rename('SoilMoisture')
compositeImg = compositeRaw.addBands(soilmoisture).addBands(forestAge)
# check the band names in the composite
compositeBandNames = compositeImg.bandNames()
print(colored('Band in composite:\n', 'blue', attrs=['bold']),compositeBandNames.getInfo())

Band in Forest age data:
 ['forest_age_TC000', 'forest_age_TC010', 'forest_age_TC020', 'forest_age_TC030', 'forest_age_TCloss_intensity', 'forest_age_LastTimeTCloss_std']
Band in composite:
 ['Aridity_Index', 'CHELSA_Annual_Mean_Temperature', 'CHELSA_Annual_Precipitation', 'CHELSA_Isothermality', 'CHELSA_Max_Temperature_of_Warmest_Month', 'CHELSA_Mean_Diurnal_Range', 'CHELSA_Mean_Temperature_of_Coldest_Quarter', 'CHELSA_Mean_Temperature_of_Driest_Quarter', 'CHELSA_Mean_Temperature_of_Warmest_Quarter', 'CHELSA_Mean_Temperature_of_Wettest_Quarter', 'CHELSA_Min_Temperature_of_Coldest_Month', 'CHELSA_Precipitation_Seasonality', 'CHELSA_Precipitation_of_Coldest_Quarter', 'CHELSA_Precipitation_of_Driest_Month', 'CHELSA_Precipitation_of_Driest_Quarter', 'CHELSA_Precipitation_of_Warmest_Quarter', 'CHELSA_Precipitation_of_Wettest_Month', 'CHELSA_Precipitation_of_Wettest_Quarter', 'CHELSA_Temperature_Annual_Range', 'CHELSA_Temperature_Seasonality', 'Depth_to_Water_Table', 'EarthEnvCloudCover_MOD

In [5]:
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

### Check the data structure of the paramter tables

In [ ]:
# load the grid searh table from R
parameterTable = pd.read_csv('Data/GridSearchResultsGEE/Wood_Density_Buffer_Based_Subsample_Grid_Search_Seed_0.csv', float_precision='round_trip')
# show the structure by head function
print(colored('The head of the table: \n', 'blue', attrs=['bold']))
parameterTable.head()

The head of the table: 



,Unnamed: 0,Mean_R2,StDev_R2,ModelName,numberOfTrees,variablesPerSplit,minLeafPopulation,bagFraction,maxNodes,type
0,0,0.49583,0.029477,GridSeach_Model_15_2_90,250,15,2,0.632,90,Classifier.smileRandomForest


In [7]:
# define the list of predictors
predictorsSelected = ["Aridity_Index",
                      "CHELSA_Annual_Mean_Temperature",
                      "CHELSA_Annual_Precipitation",
                      "CHELSA_Isothermality",
                      "CHELSA_Max_Temperature_of_Warmest_Month",
                      "CHELSA_Mean_Diurnal_Range",
                      "CHELSA_Mean_Temperature_of_Coldest_Quarter",
                      "CHELSA_Mean_Temperature_of_Driest_Quarter",
                      "CHELSA_Mean_Temperature_of_Warmest_Quarter",
                      "CHELSA_Mean_Temperature_of_Wettest_Quarter",
                      "CHELSA_Min_Temperature_of_Coldest_Month",
                      "CHELSA_Precipitation_Seasonality",
                      "CHELSA_Precipitation_of_Coldest_Quarter",
                      "CHELSA_Precipitation_of_Driest_Month",
                      "CHELSA_Precipitation_of_Driest_Quarter",
                      "CHELSA_Precipitation_of_Warmest_Quarter",
                      "CHELSA_Precipitation_of_Wettest_Month",
                      "CHELSA_Precipitation_of_Wettest_Quarter",
                      "CHELSA_Temperature_Annual_Range",
                      "CHELSA_Temperature_Seasonality",
                      "Depth_to_Water_Table",
                      "EarthEnvCloudCover_MODCF_interannualSD",
                      "EarthEnvCloudCover_MODCF_intraannualSD",
                      "EarthEnvCloudCover_MODCF_meanannual",
                      "EarthEnvTopoMed_Eastness",
                      "EarthEnvTopoMed_Elevation",
                      "EarthEnvTopoMed_Northness",
                      "EarthEnvTopoMed_ProfileCurvature",
                      "EarthEnvTopoMed_Roughness",
                      "EarthEnvTopoMed_Slope",
                      "EarthEnvTopoMed_TopoPositionIndex",
                      "SG_Absolute_depth_to_bedrock",
                      "WorldClim2_SolarRadiation_AnnualMean",
                      "WorldClim2_WindSpeed_AnnualMean",
                      "WorldClim2_H2OVaporPressure_AnnualMean",
                      "NDVI",
                      "EVI",
                      "Lai",
                      "Fpar",
                      "Npp",
                      "Tree_Density",
                      "PET",
                      "SG_Clay_Content_0_100cm",
                      "SG_Coarse_fragments_0_100cm",
                      "SG_Sand_Content_0_100cm",
                      "SG_Silt_Content_0_100cm",
                      "SG_Soil_pH_H2O_0_100cm",
                      "LandCoverClass_Cultivated_and_Managed_Vegetation",
                      "LandCoverClass_Urban_Builtup",
                      "Human_Disturbance",
                      "PresentTreeCover",
                      "Nitrogen",
                      "CanopyHeight",
                      "cropland",
                      "grazing",
                      "pasture",
                      "rangeland",
                      "Fire_Frequency",
                      "cnRatio",
                      "Cation",
                      "SoilMoisture",
                      "ForestAge"]
# show the predictors
print(colored('The predictors are:', 'blue', attrs=['bold']),predictorsSelected)

The predictors are: ['Aridity_Index', 'CHELSA_Annual_Mean_Temperature', 'CHELSA_Annual_Precipitation', 'CHELSA_Isothermality', 'CHELSA_Max_Temperature_of_Warmest_Month', 'CHELSA_Mean_Diurnal_Range', 'CHELSA_Mean_Temperature_of_Coldest_Quarter', 'CHELSA_Mean_Temperature_of_Driest_Quarter', 'CHELSA_Mean_Temperature_of_Warmest_Quarter', 'CHELSA_Mean_Temperature_of_Wettest_Quarter', 'CHELSA_Min_Temperature_of_Coldest_Month', 'CHELSA_Precipitation_Seasonality', 'CHELSA_Precipitation_of_Coldest_Quarter', 'CHELSA_Precipitation_of_Driest_Month', 'CHELSA_Precipitation_of_Driest_Quarter', 'CHELSA_Precipitation_of_Warmest_Quarter', 'CHELSA_Precipitation_of_Wettest_Month', 'CHELSA_Precipitation_of_Wettest_Quarter', 'CHELSA_Temperature_Annual_Range', 'CHELSA_Temperature_Seasonality', 'Depth_to_Water_Table', 'EarthEnvCloudCover_MODCF_interannualSD', 'EarthEnvCloudCover_MODCF_intraannualSD', 'EarthEnvCloudCover_MODCF_meanannual', 'EarthEnvTopoMed_Eastness', 'EarthEnvTopoMed_Elevation', 'EarthEnvTopoM

### After you run the chunk below, please check the status of the maping tasks on your Google earth engine

In [ ]:
# define a loop through the seed list
seedList = np.arange(0, 200, 1).tolist()
print(colored('The models are:', 'blue', attrs=['bold']),seedList)
print(colored('Model is running:\nWith paramter sets:', 'blue', attrs=['bold']))
# for seed in seedList: range(0,len(seedList))
for seed in seedList:
    # load the points data in shape file format
    woodDensityPoints = ee.FeatureCollection('users/leonidmoore/WoodDensityProject/TrainTables/Wood_Density_BufferZone_Subsampled_Train_table_Seed_'+str(seed))
    # check the information of the FeatureCollection
    # print(woodDensityTable.first().getInfo())
    # extract the environment covariates
    extractedVariableTable = compositeImg.select(predictorsSelected).reduceRegions(collection = woodDensityPoints,
                                                                                     reducer = ee.Reducer.first())
    # show the str of the extracted data
    # print(extractedVariableTable.first().getInfo())
    # exclude the rows with null valus
    trainTable = extractedVariableTable.filter(ee.Filter.notNull(predictorsSelected))
    # print(trainTable.size().getInfo())
    parameterTable = pd.read_csv('Data/GridSearchResultsGEE/Wood_Density_Buffer_Based_Subsample_Grid_Search_Seed_'+str(seed)+'.csv', float_precision='round_trip')
    # not recomend to run the code below
    # print(parameterTable.head())
    # extract the paramters
    variablesPerSplitVal = int(parameterTable['variablesPerSplit'].iat[0]) # mtry
    minLeafPopulationVal = int(parameterTable['minLeafPopulation'].iat[0]) # minrow
    maxNodesVal = int(parameterTable['maxNodes'].iat[0]) # mac depth
    print('seed',seed,variablesPerSplitVal,minLeafPopulationVal,maxNodesVal)
    # define the random forest classifier
    rfClassifier = ee.Classifier.smileRandomForest(numberOfTrees = 250,
                                                   variablesPerSplit = variablesPerSplitVal, # mtry
                                                   minLeafPopulation = minLeafPopulationVal, # minrow
                                                   maxNodes = maxNodesVal, # max depth
                                                   bagFraction = 0.632,
                                                   seed = seed).setOutputMode('REGRESSION')
    trainedClassifier = rfClassifier.train(features = trainTable,
                                           classProperty = 'WdDnsty',
                                           inputProperties = predictorsSelected)
    # execute the prediction to generate the map
    predictedWoodDensityMap = compositeImg.classify(trainedClassifier)
    # print(predictedWoodDensityMap.getInfo())
    predictionExport = ee.batch.Export.image.toAsset(image = predictedWoodDensityMap,
                                                     description = '20230329_Wood_Density_Map_To_Asset_'+str(seed),
                                                     assetId = 'users/leonidmoore/WoodDensityProject/PredictedMaps/Predicted_WoodDensity_BufferBased_Mean_Map_with_Seed_'+str(seed),
                                                     region = unboundedGeo,
                                                     crs = 'EPSG:4326',
                                                     crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                     maxPixels = 1e13)

    # print(predictionExportAsset)
    # start the export task
    predictionExport.start()
    # show the task status
    predictionExport.status()

The models are: [0, 1, 2, 3, 4, 5]
Model is running:
With paramter sets:
seed 0 15 2 90
seed 1 18 14 100
seed 2 18 8 90
seed 3 21 2 100
seed 4 18 10 100
seed 5 21 8 80
